# RNN for Sentiment Analysis (IMDB movie Reviews)

### Step 0: Setup and Imports

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

2025-10-03 18:47:09.933896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759517229.956612     320 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759517229.964917     320 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Step 1: Load and Preprocess Data

In [2]:
vocab_size=20000 # keep top words
max_len = 200 # Pad/truncate reviews to this length

(x_train,y_train),(x_test,y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

# Pad sequences to fixed length
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len, padding='post',truncating='post')
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len, padding='post',truncating='post')


### Step 2: Define Model

In [3]:
# Step 2A: Simple RNN baseline
inputs = tf.keras.Input(shape=(max_len,),name='tokens')
x = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, name='emb')(inputs)
x = tf.keras.layers.SimpleRNN(64, return_sequences=False,name='rnn')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='sentiment')(x)

rnn_model = tf.keras.Model(inputs, outputs, name='imdb_rnn_baseline')
rnn_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
rnn_model.summary()

I0000 00:00:1759517237.226715     320 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1759517237.227364     320 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "imdb_rnn_baseline"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tokens (InputLayer)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ emb (Embedding)                 │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (SimpleRNN)                 │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sentiment (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,572,417 (9.81 MB)

 Trainable params: 2,572,417 (9.81 MB)

 Non-trainable params: 0 (0.00 B)

### Step 3: Train & Record Accuracy per Epoch

In [4]:
hist_rnn = rnn_model.fit(
    x_train, y_train,
    epochs=10,
    validation_split=.1,
    batch_size=128,
    verbose=2
)
test_loss, test_acc = rnn_model.evaluate(x_test, y_test,verbose=0)
print(f"SimpleRNN Test ACC:{test_acc}")

Epoch 1/10


I0000 00:00:1759517240.147460     370 service.cc:148] XLA service 0x798544004970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759517240.147495     370 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1759517240.147500     370 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1759517240.350992     370 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1759517241.173372     370 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


176/176 - 8s - 44ms/step - accuracy: 0.5078 - loss: 0.6947 - val_accuracy: 0.4996 - val_loss: 0.6946
Epoch 2/10
176/176 - 3s - 17ms/step - accuracy: 0.6556 - loss: 0.5830 - val_accuracy: 0.4996 - val_loss: 0.7613
Epoch 3/10
176/176 - 3s - 17ms/step - accuracy: 0.7488 - loss: 0.4010 - val_accuracy: 0.5132 - val_loss: 0.8989
Epoch 4/10
176/176 - 3s - 17ms/step - accuracy: 0.7836 - loss: 0.3425 - val_accuracy: 0.5128 - val_loss: 1.0699
Epoch 5/10
176/176 - 3s - 17ms/step - accuracy: 0.7963 - loss: 0.3339 - val_accuracy: 0.5032 - val_loss: 1.0840
Epoch 6/10
176/176 - 3s - 17ms/step - accuracy: 0.8104 - loss: 0.3138 - val_accuracy: 0.4996 - val_loss: 1.0908
Epoch 7/10
176/176 - 3s - 17ms/step - accuracy: 0.8306 - loss: 0.2899 - val_accuracy: 0.5072 - val_loss: 1.1431
Epoch 8/10
176/176 - 3s - 17ms/step - accuracy: 0.8404 - loss: 0.2844 - val_accuracy: 0.5168 - val_loss: 1.1193
Epoch 9/10
176/176 - 3s - 17ms/step - accuracy: 0.8724 - loss: 0.2405 - val_accuracy: 0.5104 - val_loss: 1.2579
Epo